In [1]:
import tkinter as tk
from tkinter import ttk
from tkinter import messagebox
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib.backends.backend_tkagg import FigureCanvasTkAgg
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score
from nltk.sentiment.vader import SentimentIntensityAnalyzer
import nltk

In [2]:
apps_df = pd.read_csv("C:/Users/Ayush Singh/Downloads/google_play_store_data/googleplaystore.csv")
reviews_df = pd.read_csv("C:/Users/Ayush Singh/Downloads/google_play_store_data/googleplaystore_user_reviews.csv")

In [3]:
#Step 2 : Data Cleaning
apps_df = apps_df.dropna(subset=['Rating'])
apps_df = apps_df.dropna(subset=['Rating'])
for column in apps_df.columns :
    apps_df[column].fillna(apps_df[column].mode()[0])
apps_df = apps_df.drop_duplicates()
apps_df = apps_df[apps_df['Rating'] <= 5]
reviews_df = reviews_df.dropna(subset=['Translated_Review'])

In [4]:
merged_df = pd.merge(apps_df, reviews_df, on='App', how='inner')

In [5]:
apps_df['Reviews'] = apps_df['Reviews'].astype(int)
apps_df['Installs'] = apps_df['Installs'].str.replace(',', '').str.replace('+', '').astype(int)
apps_df['Price'] = apps_df['Price'].str.replace('$', '').astype(float)

In [6]:
def convert_size(size):
    if 'M' in size:
        return float(size.replace('M', ''))
    elif 'k' in size:
        return float(size.replace('k', '')) / 1024
    else:
        return np.nan

In [7]:
apps_df['Size'] = apps_df['Size'].apply(convert_size)

In [8]:
apps_df['Log_Installs'] = np.log1p(apps_df['Installs'])
apps_df['Log_Reviews'] = np.log1p(apps_df['Reviews'])

In [9]:
def rating_group(rating):
    if rating >= 4:
        return 'Top rated'
    elif rating >= 3:
        return 'Above average'
    elif rating >= 2:
        return 'Average'
    else:
        return 'Below average'

apps_df['Rating_Group'] = apps_df['Rating'].apply(rating_group)

In [10]:
apps_df['Revenue'] = apps_df['Price'] * apps_df['Installs']

In [11]:
sia = SentimentIntensityAnalyzer()
reviews_df['Sentiment_Score'] = reviews_df['Translated_Review'].apply(lambda x: sia.polarity_scores(str(x))['compound'])

In [12]:
apps_df['Last Updated'] = pd.to_datetime(apps_df['Last Updated'], errors='coerce')
apps_df['Year'] = apps_df['Last Updated'].dt.year